In [2]:
import pandas as pd
import numpy as np
import plotly
import re
import math
import textblob
from textblob import TextBlob
from textblob import Blobber
from textblob.wordnet import Synset

In [3]:
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Julie\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
%matplotlib inline

# To do: 
- Separate by "or" and ";" also!
- *Polarity check before meaning check because of stop words processing should be better 'v' (kinda done)*

#### Reading the pre-cleaned data

In [5]:
flower_data = pd.read_csv("../Data/General/flower_cleaned.csv", error_bad_lines=False, delimiter=";")

In [6]:
flower_data.sample(10)

,Flower,Meaning,AfinnScore,SimilarIndex40,SimilarIndex30,SimilarIndex20
83,Cherry blossom,a good education,3,[],"[148, 168, 315]","[88, 92, 117, 118, 148, 151, 168, 181, 197, 19..."
25,Aster,daintiness,0,[],[],[117]
231,Pitch pine blossom,philosophy,0,[],[],[]
227,Peony,honor (in china),2,[],[229],"[229, 256, 260, 286, 290, 329]"
90,Chestnut,do me justice,2,[],[],[]
307,Thistle,nobility,0,[],[],[]
325,Tulip [white],sincerity,2,[],[],[]
205,Moonflower,dreaming of love,3,"[206, 208, 236, 245, 250, 253, 256, 274, 286, ...","[206, 208, 218, 236, 245, 250, 253, 256, 274, ...","[206, 208, 211, 216, 218, 221, 222, 227, 229, ..."
153,Gladiolus,conviction,0,[],[],[]
152,Gladiolus,honor,2,"[184, 196, 260, 329]","[184, 196, 227, 260, 329]","[176, 184, 196, 227, 259, 260, 310, 329]"


---
## - NLP towards network graph building

---
#### Using AFINN DICTIONARY for sentiment analysis

In [8]:
path = "../External/afinn-master/afinn/data/AFINN-111.txt"
word_data_afinn = pd.read_csv(path, delimiter="\t", header=None, names=["Word", "Score"])

In [9]:
word_data_afinn.sample(n = 10)

,Word,Score
92,agonized,-3
525,crisis,-3
1190,hoax,-2
1128,gullibility,-2
2254,tout,-2
257,bereaves,-2
2193,suspecting,-1
234,banned,-2
145,apologize,-1
1637,opportunity,2


In [10]:
def afinn_score(sentence):
    split_sentence = sentence.split(" ")
    total_score = word_data_afinn.Score[word_data_afinn.Word.isin(split_sentence)].sum()
    return total_score

In [11]:
#flower_data['AfinnScore'] = flower_data.Meaning.apply(afinn_score)

In [1]:
flower_data.sample(n=10)

NameError: name 'flower_data' is not defined

---
#### Using TEXTBLOB for semantic meaning comparison

In [13]:
wiki = TextBlob("Python is a high-level, general-purpose programming language.")
wiki.tags

[('Python', 'NNP'),
 ('is', 'VBZ'),
 ('a', 'DT'),
 ('high-level', 'JJ'),
 ('general-purpose', 'JJ'),
 ('programming', 'NN'),
 ('language', 'NN')]

In [14]:
wiki.sentiment
TextBlob("my regrets follow you to the grave").sentiment

Sentiment(polarity=0.0, subjectivity=0.0)

Sentiment(polarity=-0.1, subjectivity=0.2)

In [15]:
a = wiki.words
a

WordList(['Python', 'is', 'a', 'high-level', 'general-purpose', 'programming', 'language'])

In [16]:
# Programming
a[5].lemmatize('v')

'program'

Get the highest comparison score for each synsets of a word
Limit to nouns words

In [8]:
textblob.Word("like").definitions
textblob.Word("Love").definitions

neverendsyn = textblob.Word("like").synsets
lovesyn = textblob.Word("Love").synsets

print("Similarity:", lovesyn[6].path_similarity(neverendsyn[0]))

['a similar kind',
 'a kind of person',
 'prefer or wish to do something',
 'find enjoyable or agreeable',
 'be fond of',
 'feel about or towards; consider, evaluate, or regard',
 'want to have',
 'resembling or similar; having the same or some of the same characteristics; often used in combination',
 'equal in amount or value',
 'having the same or similar characteristics',
 'conforming in every respect']

['a strong positive emotion of regard and affection',
 'any object of warm affection or devotion; ',
 'a beloved person; used as terms of endearment',
 'a deep feeling of sexual desire and attraction',
 'a score of zero in tennis or squash',
 'sexual activities (often including sexual intercourse) between two people',
 'have a great affection or liking for',
 'get pleasure from',
 'be enamored or in love with',
 'have sexual intercourse with']

In [9]:
textblob.Word("thriftiness").definitions
textblob.Word("economy").definitions

['frugality in the expenditure of money or resources']

['the system of production and distribution and consumption',
 'the efficient use of resources',
 'frugality in the expenditure of money or resources',
 'an act of economizing; reduction in cost']

In [18]:
stop_words = set(stopwords.words('english'))

In [19]:
def clean_stopwords(sentence, stopwords):
    clean_sentence = []
    for word in sentence.split(" "):
        if word not in stopwords:
            clean_sentence.append(word)
    return " ".join(clean_sentence)

In [20]:
clean_stopwords("my regrets follow you to the grave", stop_words)
ablob = TextBlob("Love between two woman.")
ablob.words

'regrets follow grave'

WordList(['Love', 'between', 'two', 'woman'])

In [21]:
# Returns the highest similarity score between two sentences (with 1 being very similar, 0 being different).
def compare_meaning_sentence(sentence1, sentence2, stopwords):
    blob1 = TextBlob(clean_stopwords(sentence1, stopwords))
    blob2 = TextBlob(clean_stopwords(sentence2, stopwords))
    nb_words = len(blob1.words) + len(blob2.words)
    
    similarity_score = 0
    for word1 in blob1.words:
        for word2 in blob2.words:
            similarity_score += compare_meaning_word(word1, word2)
    return similarity_score / nb_words
                             
# Returns the highest similarity score between two words (with 1 being very similar, 0 being different).                             
def compare_meaning_word(word1, word2):
    highest_similarity_score = 0
    
    for synset1 in word1.synsets:
        for synset2 in word2.synsets:
            similarity_score = synset1.path_similarity(synset2)
            
            if(similarity_score != None and similarity_score > highest_similarity_score):
                #print(synset1, synset2, similarity_score)
                highest_similarity_score = similarity_score
                
    return highest_similarity_score



In [22]:
# Builds the edge network connection with respect to sentences similarities.
# The similarity_threshold refers to the lower limit of similarity score that is considered similar.
def index_flower_similarity(similarity_threshold, stopwords = stop_words):
    similarities = []
    
    for i in range(0, len(flower_data)):
        similar_indexes = []
        for j in range(i+1, len(flower_data)):
            #print("indexes:", i, j)
            score = compare_meaning_sentence(flower_data['Meaning'].iloc[i], flower_data['Meaning'].iloc[j], stop_words)
            
            if score > similarity_threshold:
                similar_indexes.append(j)
                
        similarities.append(similar_indexes)
    return similarities

#### Adding to the csv the similarity nodes with a threshold of 0.4

In [23]:
#network_links = index_flower_similarity(0.4)
#flower_data['SimilarIndex40'] = pd.Series(network_links)
#flower_data.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)

In [24]:
flower_data.head(10)

,Flower,Meaning,AfinnScore,SimilarIndex40
0,Acacia,secret love,3,"[145, 181]"
1,Acanthus,art,0,[]
2,Aconite,misanthropy,0,[]
3,Agrimony,thankfulness,0,[]
4,Aloe,grief,-2,"[112, 201, 263, 293]"
5,Almond,promise,1,"[100, 221, 328]"
6,Amaranth (Globe),immortal love,5,"[20, 62, 147, 181, 250]"
7,Amaranth,immortality,0,[]
8,Amaryllis,pride,0,[]
9,Ambrosia,love is reciprocated,3,"[13, 20, 39, 44, 62, 67, 94, 98, 106, 115, 120..."


#### Adding to the csv the similarity nodes with a threshold of 0.3

In [26]:
network_links = index_flower_similarity(0.3)
flower_data['SimilarIndex30'] = pd.Series(network_links)

FileNotFoundError: [Errno 2] No such file or directory: 'General/flower_cleaned.csv'

In [30]:
flower_data.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)

#### Adding to the csv the similarity nodes with a threshold of 0.2

In [31]:
network_links = index_flower_similarity(0.2)
flower_data['SimilarIndex20'] = pd.Series(network_links)
flower_data.to_csv("../Data/General/flower_cleaned.csv", sep=";", index = False)